# A Simple Feed Forward Neural Network from Scratch

(C) 2023 by [Damir Cavar](http://damir.cavar.me/)


In [1]:
import numpy as np

In [9]:
x = np.array([2, 3, 4, 5, 6, 7])
num_features = 5
W = np.random.rand(len(x), num_features)
b = np.random.rand(num_features)

In [27]:
x_reshaped = x.reshape(len(x), 1)
print(x_reshaped.T)
print(x_reshaped)


[[2 3 4 5 6 7]]
[[2]
 [3]
 [4]
 [5]
 [6]
 [7]]


In [8]:
W

array([[0.46075016, 0.90433791, 0.37406317, 0.41519262, 0.33529596],
       [0.09541445, 0.56952051, 0.64697407, 0.22125264, 0.82742156],
       [0.73346318, 0.03653096, 0.73059091, 0.66508867, 0.17072621],
       [0.3663648 , 0.78451048, 0.50580827, 0.02198189, 0.07921299],
       [0.38292082, 0.54110659, 0.26658554, 0.62745405, 0.87700375],
       [0.7966569 , 0.90560272, 0.26297394, 0.48049441, 0.19984668]])

In [10]:
b

array([0.94950631, 0.45440971, 0.33162325, 0.40384779, 0.71634556])

In [31]:
z = x_reshaped.T @ W + b

In [32]:
print(z)

[[17.16569332  7.11120556 13.62192661  7.31092348 12.5430463 ]]


In [39]:
def relu(x):
    return np.maximum(0,x)

In [44]:
a = relu(z)
a

array([[17.16569332,  7.11120556, 13.62192661,  7.31092348, 12.5430463 ]])

In [45]:
U = np.random.rand(len(a[0]), 1)

In [46]:
U

array([[0.28574536],
       [0.92975633],
       [0.29562202],
       [0.5270413 ],
       [0.30204148]])

In [50]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [58]:
z2 = a @ U
print(z2)
sigmoid(z2[0][0])

[[23.18532608]]


0.9999999999147409